# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Изучение общей информации

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Файл содержит 21525 строк записей в 12 столбцах. Есть пропуски в данных days_employed, total_income.  В столбце days_employed данные явно не соотвествуют заявленным (стаж работы в днях), т.к. кроме того что числа вещественные, они еще и отрицательные. Возможно данные пострадали при каком-то форматировании, например при переводе из дат в числа. Столбец total_income также содержит вещественные числа, но это выглядит натурально, т.к. вероятно ежемесячный доход высчитывался как средний по данным за определенный период.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Посмотрим на пропущенные записи в столбцах days_employed и total_income

In [4]:
data[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [5]:
data[data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [6]:
data[(data['days_employed'].isna() == True) & (data['total_income'].isna() == True)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Пропуски в обоих столбцах в одних и тех же строках, всего 2174 строки, что составляет около 10% записей. Данные столбца 'days_employed' не пригодятся нам в нашем исследовании, тем более в нем находятся недостоверные данные.
Удаляем столбец days_employed.

In [7]:
data.drop('days_employed', axis=1, inplace=True)
data.head(5)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


 Исследуем дальше. Пропуски в столбце 'total_income' заполнены значением NaN, всего 2174 значения. Тип пропущенных данных количественный. Причины возникновения пропусков неизвестны, но имеют связь с отсутствующими данными стажа работы. Можно предположить что это безработные или клиенты еще не работавшие, но  у них разный возарст и указаны различные типы занятости. Вероятно это просто клиенты не предоставлявшие сведения/справки с работы о доходах и стаже. Или данные были утеряны при выгрузках. Можно применить замену средним или  медианным значением. Посмотрим на данные столбца:
</div>

In [9]:
data['total_income'].describe()

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64

Данные имеют большой разброс и выделяющиеся значения, максимальное значение 2265604 в 10 раз превышает значение 203435 на 75%. Заполним пропуски медианным значением.   

In [10]:
total_income_median = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(value=total_income_median)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB



Проверим другие столбцы на пропуски /скрытые пропуски.
 

In [11]:
columns_name = data.columns.tolist()
for column in columns_name:
    if column == 'total_income':
        continue
    list_unique = sorted(data[column].unique())
    print(column)
    print(list_unique)
    print(data[column].value_counts()) 
    print()

children
[-1, 0, 1, 2, 3, 4, 5, 20]
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

dob_years
[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: do

Обнаружены аномальные данные в столбце 'children': -1 и 20. Будут обработаны ниже в п.3.
Скрытые пропуски в столбце 'dob_years' - возраст некоторых клиентов заполнен нулями. 
Данные в столбце количественные, заменим нулевые значения средним возрастом.

In [12]:
dob_years_mean = int(data['dob_years'].mean())
data.loc[data['dob_years'] == 0, 'dob_years'] = dob_years_mean
data['dob_years'].value_counts()

35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Обнаруженные дубликаты столбца 'education' будут обработаны в п.2.3, столбца 'purpose' - в п.2.4.
Столбец 'gender' - одно значение заполнено как 'XNA'. Данные столбца категориальные, принимают значения 'F' или 'M', заменим 'XNA' на 'F', как наиболее часто встечающееся (66%) 

In [13]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'
data['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

**Вывод**

 Обнаружены и обработаны явные пропуски в столбцах 'days_employed' и 'total_income' заполненные значениями NaN. Тип пропущенных данных количественный. Пропущенные значения 'total_income' заполнили медианным значением. Кроме того обнаружены скрытые пропуски в столбце 'dob_years', возраст некоторых клиентов был заполнен нулями, заменили на значение среднего возраста. Обработано ошибочное значение столбца 'gender'. Выявлены дубликаты и аномалии в столбцах  'education', 'purpose', 'children' - будут обработаны далее.

### Замена типа данных

In [14]:
data['total_income'] = data['total_income'].astype('int')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

Заменили вещественный тип данных на целочисленный в столбце total_income. В таблице больше нет вещественных данных. Использовался метод astype(), позволяющий устанавливать нужный тип данных.

### Обработка дубликатов

In [15]:
#Проверяем на наличие дубликатов категориальные переменные методом value_counts(), возвращающим уникальные значения и их частоту
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [16]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [17]:
data['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

In [18]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [19]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Дубликаты выявлены в столбце education, связаны с написанием символов в разном регистре. Приведем записи столбца к нижнему регистру методом str.lower()

In [20]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

 Проверим строки на полные дубликаты методом duplicated():

In [21]:
print('Количество дубликатов: ', data.duplicated().sum())
data[data.duplicated() == True].head()

Количество дубликатов:  71


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
3290,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
4182,1,34,высшее,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу


 Обнаружен 71 дубликат. Дубликаты могли образоваться при копировании, переносе данных, при сбоях сохранения. Также весьма вероятно появление дубликатов при удалении столбцов содержавших дополнительные данные делавшие записи уникальными или при внесении в базу неполных данных, заполнении пропущенных значений  значениями по умолчанию и др. Дубликаты могут искажать получаемые результаты, снижать точность рассчетов, при большом количестве увеличивают размер таблиц и время обработки данных. Удалим выявленные дубликаты методом drop_duplicates().

In [22]:
data = data.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов после обработки: ', data.duplicated().sum())
print()
data.info()
data.head()

Количество дубликатов после обработки:  0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 11 columns):
children            21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Проверили таблицу на полные дубликаты методом duplicated(), обануруженные 71 дубликат удалили с обновлением индексов. Дубликаты могли появиться в таблице в результате удаления других столбцов с данными, делавшими строки уникальными, внесения неполных данных или заполнения пропусков значеними по умолчанию, также возможны технически причины - проблемы при копировании, переносе данных, при сбоях сохранения и др. Проверили категориальные переменные методом value_counts() на наличие дубликатов. Выявлены дубликаты в столбце 'education' из-за  написания симфолов в разном регистре. Обработали их методом приведения символов к нижнему регистру. Также наблюдаются дубликаты в столбце 'purpose', вероятно клиенты в свободной форме описывали цели получения кредита, из-за чего количство уникальных запсией оказалось значительным. Обработаем эти дубликаты с помощью лемматизации и стемминга.

### Лемматизация

In [23]:
#Определим ключевые слова целей кредитования
#Сохраняем в переменную purposes уникальные значения целей кредитования
purposes = data['purpose'].unique()

#Обрабатываем полученные значения для получения лемм
from pymystem3 import Mystem
m = Mystem()
purpose_lem = []
for purpose in purposes:
    purpose_lem += m.lemmatize(purpose)
    
print(purpose_lem)

['покупка', ' ', 'жилье', '\n', 'приобретение', ' ', 'автомобиль', '\n', 'дополнительный', ' ', 'образование', '\n', 'сыграть', ' ', 'свадьба', '\n', 'операция', ' ', 'с', ' ', 'жилье', '\n', 'образование', '\n', 'на', ' ', 'проведение', ' ', 'свадьба', '\n', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n', 'покупка', ' ', 'недвижимость', '\n', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n', 'покупка', ' ', 'жилой', ' ', 'недвижимость', '\n', 'строительство', ' ', 'собственный', ' ', 'недвижимость', '\n', 'недвижимость', '\n', 'строительство', ' ', 'недвижимость', '\n', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n', 'строительство', ' ', 'жилой', ' ', 'недвижимость', '\n', 'жилье', '\n', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n', 'автомобиль', '\n', 'заниматься', ' ', 'образование', '\n', 'сделка', ' '

In [24]:
#Анализируем частоту встречаемости полученных лемм
from collections import Counter
repetition_rate = Counter(purpose_lem)
print(repetition_rate)

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


In [25]:
#Примеянем стемминг для выявления основ полученных слов
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')
stemmed_words = []
for word in repetition_rate.keys():
    stemmed_words.append(russian_stemmer.stem(word))
    
print(stemmed_words)

['покупк', ' ', 'жил', '\n', 'приобретен', 'автомобил', 'дополнительн', 'образован', 'сыгра', 'свадьб', 'операц', 'с', 'на', 'проведен', 'для', 'сем', 'недвижим', 'коммерческ', 'жил', 'строительств', 'собствен', 'подержа', 'сво', 'со', 'занима', 'сделк', 'подержа', 'получен', 'высок', 'профильн', 'сдач', 'ремонт']


**Вывод**

С помощью лемматизации и стемминга определили основные категории целей кредитования и их стемы:
1. недвижимость - 'недвижим', 'жил'
2. автомобиль - 'автомобил'
3. образование - 'образован'
4. свадьба - 'свадьб'
Будем использовать их при создании функции категоризации целей кредитования.

### Категоризация данных

In [26]:
#Функция для категризации целей кредитования
def purpose_category(purpose):
    for word in purpose.split(' '):
        if ('недвижим' in word) or ('жил' in word):
            return 'недвижимость'
        if 'автомобил' in word:
            return 'автомобиль'
        if 'образован' in word:
            return 'образование'
        if 'свадьб' in word:
            return 'свадьба'
    return 'не определено'

In [27]:
#Добавляем в таблицу столбец с категориями целей кредитования
data['purpose_category'] = data['purpose'].apply(purpose_category)
data.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


In [28]:
#проверяем наличие не определенных категорий
data['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

Для определения зависимости между наличием детей и возвратом кредита в срок нам необходимо разделить клиентов на две группы - имеющих детей и нет. Создадим таблицу selection_child, с данными о наличии детей и случаями невозварата кредитов. Добавим столбец 'have_children', со значениями 1 - есть дети, 0 - нет.

Ранее в столбце 'children' были выявлены аномальные значения -1 и 20. -1 похоже на заглушку, на заполнение пропущенных значений или код какой-то ошибки. 20 - сложнее, непонятно, может это 2, а может 0 - в нашем анализе это будут полярные значения. Так как выяснить реальные данные мы не можем, принимаем решение не учитывать эти данные. Сделаем отбор данных для анализа зависимости от наличия детей в переменную selection_child 

In [29]:
#проверяем значения столбца 'children'
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [30]:
selection_child = data[['children','debt']]
selection_child = selection_child.loc[(selection_child['children'] >= 0) & (selection_child['children'] < 20)]
selection_child['children'].value_counts()

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

In [31]:
#Функция для категоризации клиентов по признаку наличия детей
def have_children(children):
    if children > 0:
        return 1
    return 0

In [32]:
selection_child['have_children'] = selection_child['children'].apply(have_children)
selection_child.head(10)

,children,debt,have_children
0,1,0,1
1,1,0,1
2,0,0,0
3,3,0,1
4,0,0,0
5,0,0,0
6,0,0,0
7,0,0,0
8,2,0,1
9,0,0,0


Для подготовки данных для определения зависимости между семейным положением и возвратом кредита в срок создадим таблицу selection_family и словарь статусов семейного положения:

In [33]:
selection_family = data[['family_status_id', 'debt']]
selection_family.head(10)

,family_status_id,debt
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0
5,1,0
6,0,0
7,0,0
8,1,0
9,0,0


In [34]:
family_st_dict = data[['family_status','family_status_id']]
#удаляем дубликаты
family_st_dict = family_st_dict.drop_duplicates().reset_index(drop=True)
family_st_dict.head(10)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Для определения зависимости между уровнем дохода и возвратом кредита в срок, разделим клиентов по уровню доходов на 4 категории: низкий, средний, высокий, сверхвысокий. Определим 4 группы доходов с помощью метода qcut()

In [35]:
pd.qcut(data['total_income'], q=4).value_counts()

(107623.0, 145017.0]      6415
(195813.25, 2265604.0]    5364
(20666.999, 107623.0]     5364
(145017.0, 195813.25]     4311
Name: total_income, dtype: int64

In [36]:
income_mean = data['total_income'].mean()
income_low = income_mean / 1.88
income_medium = income_mean / 1.22
income_high = income_mean / 0.85


In [37]:
#функция категоризации доходов клиентов
def income_category_set(income):
    if income <= income_low:
        return 'низкий'
    elif (income > income_low) & (income <= income_medium):
        return 'средний'
    elif (income > income_medium) & (income <= income_high):
        return 'высокий'
    return 'сверхвысокий'

In [38]:
data['income_category'] = data['total_income'].apply(income_category_set)
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_category
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,сверхвысокий
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,высокий
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,сверхвысокий
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,высокий


**Вывод**

На основании информации полученной на предыдущем этапе, разделили цели получения кредитов на 4 категории. В таком виде их будет удобно обрабатывать в дальнейших исследованиях. Также подготовили и категоризировали данные по наличию детей, по семейному положению и сгруппировали клиентов по уровню доходов.

Создадим общую функцию для обработки и рассчета зависимостей
Функция получает на вход отфильтрованные данные (selection_data), 
имя категории группирования(name_for_groupby),
список ключевых категорий для расчета зависимостей (keys),
sort - признак необходимости сортировки выходных данных

In [39]:
def data_result(selection_data, name_for_groupby, keys, sort):
    #Получаем количество клиентов имевших задолженности по возвратам кредитов в разрезе категорий
    debt_count = selection_data.groupby(name_for_groupby)['debt'].sum()
    #Получаем количество клиентов получивших кредит в разрезе категорий
    credit_count = selection_data.groupby(name_for_groupby)['debt'].count()
    #Создаем итоговую таблицу с данными для анализа
    data = []
    for i in range(len(debt_count)):
        data.append([keys[i], debt_count[i], credit_count[i]])
    
    result = pd.DataFrame(data=data, 
                            columns=[name_for_groupby, 'debt_count', 'credit_count'])
    result['bad_dabt_percent'] = (result['debt_count'] / result['credit_count']) * 100
    if sort == True:
        result = result.sort_values('bad_dabt_percent', ascending=False)
        return result
    return result

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [40]:
keys_child = sorted(selection_child['have_children'].unique())
result_child = data_result(selection_child, 'have_children', keys_child, False)
result_child

,have_children,debt_count,credit_count,bad_dabt_percent
0,0,1063,14091,7.543822
1,1,669,7240,9.240331


**Вывод**

Процент имевших задолженности по возвратам кредитов, среди клиентов с детьми: 9.24%, среди клиентов без детей 7.54%. Вывод: клиенты с детьми чаще допускают задолженности по возвратам кредитов. Очеивидно клиенты с детьми чаще сталкиваются с непредвиденными расходами и им сложнее планировать свой бюджет. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [41]:
keys_family = sorted(selection_family['family_status_id'].unique())
result_family = data_result(selection_family, 'family_status_id', keys_family, True)
result_family

,family_status_id,debt_count,credit_count,bad_dabt_percent
4,4,274,2810,9.750890
1,1,388,4151,9.347145
0,0,931,12339,7.545182
3,3,85,1195,7.112971
2,2,63,959,6.569343


In [42]:
#Добавляем в итоговую таблицу текстовые значения семейного положения
result_family.insert(0,'family_status', result_family['family_status_id'].map(family_st_dict.set_index('family_status_id')['family_status']))
result_family

,family_status,family_status_id,debt_count,credit_count,bad_dabt_percent
4,Не женат / не замужем,4,274,2810,9.750890
1,гражданский брак,1,388,4151,9.347145
0,женат / замужем,0,931,12339,7.545182
3,в разводе,3,85,1195,7.112971
2,вдовец / вдова,2,63,959,6.569343


**Вывод**

Самые дисциплинированные плательщики кредитов вдовцы и вдовы, а также разведенные, видимо они расчитывают только на себя и тщательно планируют свои расходы. Чаще других допускают просрочки оплат клиенты, которые не женатые/не замужем, а также находящиеся в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [43]:
#Отберем данные для анализа зависимости возвратов кредитов от уровня доходов
selection_income = data[['income_category', 'debt']]
selection_income.head()

,income_category,debt
0,сверхвысокий,0
1,средний,0
2,высокий,0
3,сверхвысокий,0
4,высокий,0


In [44]:
keys_income = sorted(selection_income['income_category'].unique())
#Передаем данные в основную функцию
result_income = data_result(selection_income, 'income_category', keys_income, True)
result_income

,income_category,debt_count,credit_count,bad_dabt_percent
0,высокий,647,7408,8.733801
3,средний,460,5442,8.452775
1,низкий,239,3136,7.621173
2,сверхвысокий,395,5468,7.223848


**Вывод**

Как ни странно, но самые недисциплинированные плательщики - клиенты с высокими доходами от 135000 до 195000. При этом клиенты имеющие сверхвысокий доход (более 195000) являются самыми надежными кредитополучателями. Также низкий процент просрочки оплат демонстируют клиенты с низкими доходами (менее 88000)

- Как разные цели кредита влияют на его возврат в срок?

In [45]:
#Создадим таблицу selection_purpose с данными целей кредитовния
selection_purpose = data[['purpose_category', 'debt']]
selection_purpose.head(10)

,purpose_category,debt
0,недвижимость,0
1,автомобиль,0
2,недвижимость,0
3,образование,0
4,свадьба,0
5,недвижимость,0
6,недвижимость,0
7,образование,0
8,свадьба,0
9,недвижимость,0


In [46]:
keys_purpose = sorted(selection_purpose['purpose_category'].unique())
result_purpose = data_result(selection_purpose, 'purpose_category', keys_purpose, True)
result_purpose

,purpose_category,debt_count,credit_count,bad_dabt_percent
0,автомобиль,403,4306,9.359034
2,образование,370,4013,9.220035
3,свадьба,186,2324,8.003442
1,недвижимость,782,10811,7.233373


**Вывод**

Меньше всего просрочек допускают клиенты оформляющие кредиты на недвижимость и празднование свадеб. Чаще других не вовремя платят купившие в кредит автомобиль, видимо есть у таких клиентов склонность завышать свои возможности в погогне за желанием обладать престижным авто. Также относительно высокий процент просрчек демонрируют клиенты кредитовавшиеся на образование, здесь видимо вопрос в нестабильности доходов таких клиентов.

## Шаг 4. Общий вывод

   В результате выполнения данного проекта были изучены данные по кредитованию различных категорий клиентов. В процессе подготовки к исследованию в данных были определены и заполнены пропущенные значения, заменены типы некоторых данных, произведен поиск и устранение дубликатов, выполнена категоризация данных. Необходимо обратить внимание при сборе данных на пропуски в данных days_employed, total_income. В столбце days_employed данные не соотвествуют заявленным (стаж работы в днях), т.к. числа вещественные и отрицательные. Также необходимо проверить получение данных о количестве детей, часть значений заполнены повторяющимися -1 и аномально большим числом 20.   
   Были проведены рассчеты и исследования по вопросам есть ли зависимость по возвратом кредита в срок от наличия детей, от семейного положения, от уровня доходов и от целей кредитования. 
   Определено, что клиенты с детьми чаще допускают задолженности по возвратам кредитов. Вероятно клиенты имеюще детей чаще сталкиваются с незапланированными расходами и им сложнее планировать свой бюджет. 
    При анализе зависимости между семейным положением и возвратом кредита в срок выявлено, что самые дисциплинированные плательщики кредитов вдовцы и вдовы, а также разведенные, видимо они расчитывают только на себя и тщательно планируют свои расходы. Чаще других допускают просрочки оплат клиенты, которые не женатые/не замужем, а также находящиеся в гражданском браке. 
    Интересные результаты получены при анализе зависимости между уровнем дохода просрочками оплат по кредитам. Как ни странно, но самые недисциплинированные плательщики - клиенты с высокими доходами от 135000 до 195000. При этом клиенты имеющие сверхвысокий доход (более 195000) являются самыми надежными кредитополучателями. Также низкий процент просрочки оплат демонстируют клиенты с низкими доходами (менее 88000). 
    При исследовании вопроса как разные цели кредита влияют на его возврат в срок, выявлено, что меньше всего просрочек допускают клиенты оформляющие кредиты на недвижимость и празднование свадеб. Чаще других не вовремя платят купившие в кредит автомобиль, видимо такие клиенты чаще других имеют склонность завышать свои возможности и не рассчитывать силы. Также относительно высокий процент просрчек демонрируют клиенты кредитовавшиеся на образование, здесь видимо вопрос в нестабильности доходов таких клиентов.

